First, we want to load packages and create an empty data frame to be filled. Additionally, define a starter key to collect more keys

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import time 
import datetime
final = pd.DataFrame(columns = ['caption','comments','filters','hashtags','image','photo_id','likes','time','user', 'scrape_time', 'key']) 
#chance this key will have to be changed over time - atm it's just to start collecting other keys
key = 'AQDt5LYE7lbm7Rb2P-ox-rkpSdXXT19HsvYz4Ncf_k2qcjJ3S2crfI9m-jipFD0SOtWBTAIEKkU1AozNIhN52-wfgMbUTr1biNafa9RGWnQpF6izIF8GIiNf1Dvdl_IbnR4'
counter = 0

Only run the code below if you're looking to get more keys. The number of keys you get will depend on the range

In [2]:
#collect keys
start = time.time()
result = []  
for i in range(0,1000):
    d = {}
    response = requests.post("http://www.instagim.com/ajax/",
        data='get_photos=https%3A%2F%2Fapi.instagram.com%2Fv1%2Ftags%2Fnature%2Fmedia%2Frecent%3Fmax_tag_id%' + key + '%count%3D15%26access_token%3D&next_url=https%3A%2F%2Fapi.instagram.com%2Fv1%2Ftags%2Fnature%2Fmedia%2Frecent%3Fmax_tag_id%3D%7Bnext_key%7D%26count%3D15%26access_token%3D',
        headers={
            "Accept": "application/json, text/javascript, */*; q=0.01",
            "Accept-Encoding": "gzip, deflate",
            "Accept-Language": "en-US,en;q=0.8,zh-CN;q=0.6,zh;q=0.4",
            "Connection": "keep-alive",
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "Origin": "http://www.instagim.com",
            "Referer": "http://www.instagim.com/tag/nature",
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36",
            "X-Requested-With": "XMLHttpRequest"
        },
        cookies={
            "_ga": "GA1.2.821769385.1489495570",
            "_gat": "1"
        },
    )
    data = json.loads(response.text)
    d['key'] = data['key'].encode('utf-8')
    d['post_time'] = str(time.strftime("%c"))
    result.append(d)
    time.sleep(1)
    
print 'Time elapsed: ', time.time() - start



Time elapsed:  2235.85978818


Simply writing the keys to a csv file while the photos "mature" like fine wine.

In [3]:
keys_df = pd.DataFrame(result)
print keys_df.head()
keys_df.to_csv('keys'+ str(time.strftime("%c") + '.csv'))


                                                 key                 post_time
0  AQDZCWtL4p5pt8g0aPjK1a4VZarE6_BIN-XV1gzir7JF1b...  Fri Mar 17 11:08:54 2017
1  AQCCmrdrmSWVbRItbPkr2SNMpyDiGQ3clQprj5IUXrvl4i...  Fri Mar 17 11:08:56 2017
2  AQBMVQ8EWo4oXfOyF4ps6JAh73OYuD3dRw0we0vuvoHq19...  Fri Mar 17 11:08:59 2017
3  AQAGrfUkkcynh8lq9CeJPNnhCZJr5Z4_gJZHMh9Fr2aJ5x...  Fri Mar 17 11:09:01 2017
4  AQAyN0gi88hiEB1hCnEvSAlJajVY6kgpXzoYJS5hgH1Fpg...  Fri Mar 17 11:09:03 2017


Now, we want to use yesterdays keys to actually get the data we want.


In [4]:
key = pd.read_csv('../beautifulSoup/keys_Wed_Mar_15_17_02_00_2017.csv')
key = key['0'].tolist()
key = key[0:2] # for tests


In [5]:


for key in key:
    


    response = requests.post("http://www.instagim.com/ajax/",
        data='get_photos=https%3A%2F%2Fapi.instagram.com%2Fv1%2Ftags%2Fnature%2Fmedia%2Frecent%3Fmax_tag_id%3D' + key + '%26count%3D15%26access_token%3D&next_url=https%3A%2F%2Fapi.instagram.com%2Fv1%2Ftags%2Fnature%2Fmedia%2Frecent%3Fmax_tag_id%3D%7Bnext_key%7D%26count%3D15%26access_token%3D',
        headers={
            "Accept": "application/json, text/javascript, */*; q=0.01",
            "Accept-Encoding": "gzip, deflate",
            "Accept-Language": "en-US,en;q=0.8,zh-CN;q=0.6,zh;q=0.4",
            "Connection": "keep-alive",
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "Origin": "http://www.instagim.com",
            "Referer": "http://www.instagim.com/tag/nature",
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36",
            "X-Requested-With": "XMLHttpRequest"
        },
        cookies={
            "_ga": "GA1.2.821769385.1489495570",
            "_gat": "1"
        },
    )

    data = json.loads(response.text)
    text = BeautifulSoup(data['html'], 'html.parser')
    events = text.find_all('div', {'class': "box-photo"})


    result = [] 
    for event in events:
        counter += 1
        d = {} # each event is its own dictionary so initialize it inside the loop
        try: #use try statement so code doesn't break if names are wrong
                 #remember to change this to a counter
                index = counter
                im = [x['src'] for x in event.find('div', {'class': 'photo'}).a][0]
                urllib.urlretrieve(im, str(counter) + ".jpg")
                time_post = event.find('div', {'class': "upload-time"}).get_text()
                user = event.find('div', {'class': "username"}).a.get_text()
                capt = event.find('div', {'class': "photo-description"}).get_text()
                hashtags = event.find('div', {'class': "photo-tags"}).get_text()
                filters = event.find('div', {'class': "filter-text"}).get_text()
                likes = event.find('span', {'class': "icon-thumbs-up-alt"}).get_text()
                comments = event.find('span', {'class': "icon-chat"}).get_text()
        except: 
                index = ""
                im = ""
                time_post = ""
                user = ""
                capt = ""
                hashtags = ""
                filters = ""
                likes = ""
                comments = ""
        finally:  
                d['photo_id'] = index
                d['image'] = im
                d['time'] = time_post
                d['user'] = user             
                d['caption'] = capt
                d['hashtags'] = hashtags
                d['filters'] = filters
                d['likes'] = likes
                d['comments'] = comments
                d['scrape_time'] = str(time.strftime("%c"))
                d['key'] = key
                result.append(d) # each event is own dictionary

    final = final.append(pd.DataFrame(result))


Double check that your dataframe has been created

In [6]:
final

,caption,comments,filters,hashtags,image,key,likes,photo_id,scrape_time,time,user
0,\r\n Got to get them eye dr...,2,Filter Normal,#funny #life #dog #instahub #blue #lol #frien...,https://scontent.cdninstagram.com/t51.2885-15/...,AQCz6KH-Tg8kAgZYf9gVHGQciBAyxvoJmwc8Jo3MoLQzPx...,50,1.0,Thu Mar 16 17:33:07 2017,1 day ago,@danny_l3030
1,\r\n nature🌱🌳\n#nature#su...,0,Filter Gingham,#green #photografie #sun #nature,https://scontent.cdninstagram.com/t51.2885-15/...,AQCz6KH-Tg8kAgZYf9gVHGQciBAyxvoJmwc8Jo3MoLQzPx...,38,2.0,Thu Mar 16 17:33:07 2017,1 day ago,@yourpics.de
2,"\r\n Take a photo , to have...",0,Filter Normal,#sun #vscobalkan #memories #pic #tree #vsco #...,https://scontent.cdninstagram.com/t51.2885-15/...,AQCz6KH-Tg8kAgZYf9gVHGQciBAyxvoJmwc8Jo3MoLQzPx...,31,3.0,Thu Mar 16 17:33:08 2017,1 day ago,@x.anela.xy
3,\r\n #hiking #Åland #nature...,3,Filter Ginza,#hello #finland #balance #earthporn #åland #l...,https://scontent.cdninstagram.com/t51.2885-15/...,AQCz6KH-Tg8kAgZYf9gVHGQciBAyxvoJmwc8Jo3MoLQzPx...,64,4.0,Thu Mar 16 17:33:08 2017,1 day ago,@mykola.turko
4,\r\n #pulsatillagrandis #ev...,0,Filter Normal,#pulsatillagrandis #spring #beautiful #macro ...,https://scontent.cdninstagram.com/t51.2885-15/...,AQCz6KH-Tg8kAgZYf9gVHGQciBAyxvoJmwc8Jo3MoLQzPx...,16,5.0,Thu Mar 16 17:33:08 2017,1 day ago,@giuseppe_antal
5,\r\n Jiminy #cricket #locus...,0,Filter Normal,#cricket #naturelovers #nature #grasshopper #...,https://scontent.cdninstagram.com/t51.2885-15/...,AQCz6KH-Tg8kAgZYf9gVHGQciBAyxvoJmwc8Jo3MoLQzPx...,7,6.0,Thu Mar 16 17:33:08 2017,1 day ago,@alek.kimskurt
6,\r\n 🍁This world is so Bea...,3,Filter Normal,#hope #sunsets #alive #l4l #f4f #photography ...,https://scontent.cdninstagram.com/t51.2885-15/...,AQCz6KH-Tg8kAgZYf9gVHGQciBAyxvoJmwc8Jo3MoLQzPx...,84,7.0,Thu Mar 16 17:33:08 2017,1 day ago,@gwedhiiel
7,\r\n #lisbon #lisboa #portu...,0,Filter Normal,#city #blue #himmel #architecture #lisbon #po...,https://scontent.cdninstagram.com/t51.2885-15/...,AQCz6KH-Tg8kAgZYf9gVHGQciBAyxvoJmwc8Jo3MoLQzPx...,31,8.0,Thu Mar 16 17:33:08 2017,1 day ago,@miri_berlin
8,\r\n No hay quien me deteng...,2,Filter Hefe,#marzo #momentosunicos #trail #retomandolacal...,https://scontent.cdninstagram.com/t51.2885-15/...,AQCz6KH-Tg8kAgZYf9gVHGQciBAyxvoJmwc8Jo3MoLQzPx...,14,9.0,Thu Mar 16 17:33:08 2017,1 day ago,@gmdasa
9,\r\n ¡Próxima ilustración e...,0,Filter Normal,#cute #spring #flowers #artistofinstagram #na...,https://scontent.cdninstagram.com/t51.2885-15/...,AQCz6KH-Tg8kAgZYf9gVHGQciBAyxvoJmwc8Jo3MoLQzPx...,32,10.0,Thu Mar 16 17:33:09 2017,1 day ago,@willy.and.friends


And finally write that file to a csv!

In [7]:
final.to_csv('Insta.csv', encoding = 'utf-8')